In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

class MinimumWageScraper:
    def __init__(self, url="https://en.wikipedia.org/wiki/List_of_countries_by_minimum_wage"):
        """
        Classe para realizar o web scraping de salários mínimos de países a partir de uma página da Wikipedia.
        """
        self.url = url
        self.countries = []
        self.wages = []
        self.dataframe = None

    def fetch_data(self):
        """Faz a requisição da página e obtém o conteúdo HTML."""
        response = requests.get(self.url)
        response.raise_for_status()  # Levanta um erro se a requisição falhar
        self.soup = BeautifulSoup(response.text, 'html.parser')

    def parse_table(self):
        """Extrai os dados da tabela de salários mínimos."""
        table = self.soup.find('table', {'class': 'wikitable'})
        if not table:
            raise ValueError("Tabela não encontrada na página.")

        for row in table.find_all('tr')[1:]:
            cells = row.find_all('td')
            if len(cells) > 1:  # Verifica se a linha tem as colunas esperadas
                country = cells[0].get_text(strip=True)
                wage = cells[2].get_text(strip=True)  # Pega a coluna de salários anuais nominais
                self.countries.append(country)
                self.wages.append(wage)

    def create_dataframe(self):
        """Cria um DataFrame do pandas com os dados extraídos."""
        self.dataframe = pd.DataFrame({
            'Country': self.countries,
            'Annual Nominal Wage (US$)': self.wages
        })

    def save_to_csv(self, filename):
        """Salva os dados extraídos em um arquivo CSV."""
        if self.dataframe is None:
            raise ValueError("Nenhum dado encontrado para salvar.")
        self.dataframe.to_csv(filename, index=False)
        print(f"Dados salvos em {filename}")

    def scrape(self):
        """Executa o processo completo de scraping: fetch, parse e criação do DataFrame."""
        self.fetch_data()
        self.parse_table()
        self.create_dataframe()

    def __repr__(self):
        return f"MinimumWageScraper(url='{self.url}')"


         Country Annual Nominal Wage (US$)
0    Afghanistan                       858
1        Albania                     4,637
2        Algeria                     1,777
3        Andorra                    18,253
4         Angola                       663
..           ...                       ...
196    Venezuela                     10.32
197      Vietnam                     1,591
198        Yemen                          
199       Zambia                       596
200     Zimbabwe                          

[201 rows x 2 columns]
